In [2]:
'Wind Direction - Resultant, Wind Speed - Resultant, Barometric Pressure, Outdoor Temperature', 
'Average Ambient Temperature', 'Average Ambient Pressure','Relative Humidity ',
'Solar radiation','Light scatter', 'Sample Flow Rate- CV','Sample Volume'

('Solar radiation', 'Light scatter', 'Sample Flow Rate- CV', 'Sample Volume')

In [ ]:
 'Wind Speed - Resultant, Barometric Pressure, Outdoor Temperature', 
'Average Ambient Temperature', 'Average Ambient Pressure','Relative Humidity ',
'Solar radiation'

In [2]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import statsmodels.api as sm
from sklearn.metrics import log_loss
import datascience as d
from datascience.predicates import are
sns.set(style="ticks")
%matplotlib inline
import pickle

In [5]:
#o = pd.read_csv("statedata/oregon/Oregon.csv")
#o_pm = pd.read_csv("statedata/oregon/oregon_finalpm.csv")

#c = pd.read_csv("statedata/california/California_by_county.csv")
#c_pm = pd.read_csv("statedata/california/cali_finalpm.csv")

#m = pd.read_csv("statedata/mass/Massachusetts.csv")
#m_pm = pd.read_csv("statedata/mass/mass_finalpm.csv")

#con = pd.read_csv("statedata/connecticut/Connecticut.csv")
#con_pm = pd.read_csv("statedata/connecticut/conn_finalpm.csv")

w = pd.read_csv("States.csv")
w_pm = pd.read_csv("fixed_US3_CONVERTED.csv")

#w_so2 = pd.read_csv("CA_so2.csv")
#w_no2 = pd.read_csv("CA_no2.csv")
#pd.read_csv("final_no2.csv")


#totalpts = len(w.loc[w["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"])
   #) +len(m.loc[m["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]
         #) +  len(con.loc[con["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]
                # )+len(o.loc[o["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]) 

mean_temp = np.mean(w.loc[w["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"])
      # )+np.sum(m.loc[m["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]
               # )+np.sum(w.loc[w["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]
                         #)+np.sum(con.loc[con["Parameter Name"]=="Outdoor Temperature"]["Arithmetic Mean"]))/56

mean_sol = np.mean(w.loc[w["Parameter Name"]=="Solar radiation"]["Arithmetic Mean"])
       #)+np.sum(m.loc[m["Parameter Name"]=="Solar radiation"]["Arithmetic Mean"]
               # )+np.sum(w.loc[w["Parameter Name"]=="Solar radiation"]["Arithmetic Mean"]
                        # )+np.sum(con.loc[con["Parameter Name"]=="Solar radiation"]["Arithmetic Mean"]))/56

mean_wind = np.mean(w.loc[w["Parameter Name"]=="Wind Speed - Resultant"]["Arithmetic Mean"])
       #)+np.sum(m.loc[m["Parameter Name"]=="Wind Speed - Resultant"]["Arithmetic Mean"]
        #        )+np.sum(w.loc[w["Parameter Name"]=="Wind Speed - Resultant"]["Arithmetic Mean"]
         #                )+np.sum(con.loc[con["Parameter Name"]=="Wind Speed - Resultant"]["Arithmetic Mean"]))/56
#o_temp = o.loc[o["Parameter Name"]=="Outdoor Temperature"]
mean_dew = np.mean(w.loc[w["Parameter Name"]=="Dew Point"]["Arithmetic Mean"])

mean_sul = np.mean(w.loc[w["Parameter Name"]=="Sulfation rate"]["Arithmetic Mean"])

#list(o_pm["Site_keys"])[0] in o["Site_keys"]
#(o_pm["Site_keys"])[0]
w["Date of Last Change"]

0       2010-10-13
1       2010-10-13
2       2010-10-13
3       2010-10-13
4       2016-08-18
5       2016-08-18
6       2016-08-18
7       2016-08-18
8       2010-04-25
9       2010-04-25
10      2010-04-25
11      2010-04-25
12      2010-04-25
13      2010-04-25
14      2010-04-25
15      2010-10-13
16      2010-10-13
17      2016-03-03
18      2010-04-25
19      2013-02-16
20      2013-02-16
21      2013-02-16
22      2013-02-16
23      2010-04-25
24      2013-02-16
25      2013-02-16
26      2013-02-16
27      2013-02-16
28      2010-04-25
29      2013-02-16
           ...    
9112    2010-04-25
9113    2010-04-25
9114    2010-04-25
9115    2010-04-25
9116    2010-04-25
9117    2013-06-29
9118    2010-04-25
9119    2010-04-25
9120    2013-06-29
9121    2013-02-16
9122    2013-02-16
9123    2013-02-16
9124    2013-02-16
9125    2011-01-15
9126    2011-01-15
9127    2013-05-16
9128    2013-05-16
9129    2016-05-26
9130    2010-04-25
9131    2016-04-19
9132    2016-04-19
9133    2016

In [76]:
#get mean weather shit for each 


def getweather(df, df_pm, condition):
    df_cond =  df.loc[df["Parameter Name"]==condition]
    if len(df_cond.index)>0:
        pm = df_pm["Site_keys"]
        cond = []
        for i in range(0, len(df_pm["Site_keys"]), 1):
            #if the site is inside our pm column
            if list(df_pm["Site_keys"])[i] in list(df_cond["Site_keys"]):
                #append the arithmetic mean at that index to the new list
                ind = list(df_cond["Site_keys"]).index(list(df_pm["Site_keys"])[i])
                cond.append(list(df_cond["Arithmetic Mean"])[ind])
            else:
                #append the mean condition for the state
                cond.append(np.mean(df_cond["Arithmetic Mean"]))
    else:
        if condition == "Wind Speed - Resultant":
            cond = np.repeat(mean_wind, len(df_pm.index))
        elif condition == "Outdoor Temperature":
            cond = np.repeat(mean_temp, len(df_pm.index))
        elif condition =="Solar radiation":
            cond = np.repeat(mean_sol, len(df_pm.index))
        elif condition =="Dew Point":
            cond = np.repeat(mean_dew, len(df_pm.index))
        else:
            cond = np.repeat(mean_sul, len(df_pm.index))
    return cond

#With some manipulation, picked out weather data that would result in more than 5 unique readings for at least 1 of our pollutants
weather = ['Wind Speed - Resultant', 'Outdoor Temperature', 
'Solar radiation', 'Dew Point', 'Sulfation rate']

def weatherforstate(df, df_pm):
    data = {}
    for i in weather:
        data[i] = getweather(df, df_pm, i)
    data["Site_keys"] = df_pm["Site_keys"]
    return data

#statesso2 = weatherforstate(w, w_so2)
#statesno2 = weatherforstate(w, w_no2)
statespm = weatherforstate(w, w_pm)
#conn_weather = weatherforstate(con, con_pm)
#c_weather = weatherforstate(c, c_pm)
#w_weather = weatherforstate(w, w_pm)
#m_weather = weatherforstate(m, m_pm)

In [82]:
#states_no2 = pd.DataFrame(statesno2)
#states_so2 =  pd.DataFrame(statesso2)
statespm = pd.DataFrame(statespm)
#conw =  pd.DataFrame(conn_weather)
#ww =  pd.DataFrame(w_weather)
#mw =  pd.DataFrame(m_weather)
#weatherfinal = pd.concat([mw, cw, ow, conw, ww])
#weatherfinal["index"] = np.arange(0, 198, 1)
#states_no2["index"] =  np.arange(0, len(states_no2.index), 1)
#states_so2["index"] =  np.arange(0, len(states_so2.index), 1)
statespm["index"] =  np.arange(0, len(statespm.index), 1)
#pm["index_1"] = np.arange(0, 198, 1)
#w_so2["index_1"] = np.arange(0, len(states_so2.index), 1)
#w_no2["index_1"] =  np.arange(0, len(states_no2.index), 1)
w_pm["index_1"] = np.arange(0, len(statespm.index), 1)


#fin_no2weather = pd.merge(w_no2, weatherfinal, on = "Site_keys")
#fin_pmweather.to_csv("fin_pmweather.csv", index=False)
#so2 = pd.merge(w_so2, states_so2, left_on="index_1", right_on="index", how="left")
#no2 = pd.merge(w_no2, states_no2, left_on="index_1", right_on="index", how="left")
pm = pd.merge(w_pm, statespm, left_on="index_1", right_on="index", how="left")
pm["Dew Point"].unique()

array([ 36.17081747,  37.047777  ,  33.033727  ,  37.041717  ,
        31.092125  ,  34.734375  ,  29.543299  ,  35.729857  ])

In [79]:
del pm["index"]
del pm["index_1"]
del pm["Site_keys_y"]
#del no2["index"]
#del no2["index_1"]
#del no2["Site_keys_y"]

In [80]:
#no2.to_csv("finished_no2weather.csv", index=False)
#so2.to_csv("finished_so2weather.csv", index=False)
pm.to_csv("finished_pmweather.csv", index=False)

In [83]:
pd.read_csv("finished_pmweather.csv")
#pd.read_csv("finished_so2weather.csv")
#pd.read_csv("finished_pmweather.csv")

,Unnamed: 0,Parameter Name,Site_keys_x,ppm,forest,open_land,water,wetland,saltwater/sandybeach,recreational,...,high_density_residential,medium_density_residential,low_density_residential,industrial,not_assigned,Dew Point,Outdoor Temperature,Solar radiation,Sulfation rate,Wind Speed - Resultant
0,0,Suspended particulate (TSP),[ 30.555472 -87.713599],0.051745,0.000000,0.0,0.000000,0.00,0.0,0,...,0,0.430000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
1,1,Suspended particulate (TSP),[ 31.891272 -85.13521 ],0.052419,0.130000,0.0,0.460000,0.00,0.0,0,...,0,0.160000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
2,2,Suspended particulate (TSP),[ 33.949542 -86.473042],0.051500,0.010000,0.0,0.000000,0.00,0.0,0,...,0,0.280000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
3,3,Suspended particulate (TSP),[ 33.656771 -85.824409],0.080179,0.000000,0.0,0.000000,0.00,0.0,0,...,0,0.529101,0,0.132275,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
4,4,Suspended particulate (TSP),[ 33.685659 -85.837744],0.059173,0.000000,0.0,0.000000,0.00,0.0,0,...,0,0.617284,0,0.006173,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
5,5,Suspended particulate (TSP),[ 32.83957 -86.628871],0.092320,0.060000,0.0,0.000000,0.00,0.0,0,...,0,0.320000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
6,6,Suspended particulate (TSP),[ 32.209311 -88.051961],0.074660,0.625000,0.0,0.000000,0.00,0.0,0,...,0,0.014583,0,0.014583,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
7,7,Suspended particulate (TSP),[ 34.720368 -87.819755],0.042098,0.310000,0.0,0.000000,0.00,0.0,0,...,0,0.000000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
8,8,Suspended particulate (TSP),[ 34.761756 -87.639195],0.070250,0.230769,0.0,0.000000,0.00,0.0,0,...,0,0.019231,0,0.028846,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
9,9,Suspended particulate (TSP),[ 34.716757 -87.881701],0.045600,0.000000,0.0,0.000000,0.00,0.0,0,...,0,0.000000,0,0.000000,0.0,36.170817,64.313252,0.288305,0.235677,4.763196
